In [1]:
import torch
import numpy as np
from torch import nn
from utils.model import Spice_model
from torchinfo import summary
import pandas as pd
from data_files.dataloader import MedleyDBLoader
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from utils.model import Spice_model, Spice_Encoder, Spice_Decoder
from utils.model_types import Spice_model2, Spice_Decoder_lite, Deconv_block
from utils.training_script import Trainer
from optims.loss import Huber_loss, Recons_loss, Conf_loss
from data_files.dataset import CQT_Dataset

2023-07-31 20:21:36.455332: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-31 20:21:36.493434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 20:21:37.099396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
import onnx

# Load the ONNX model
model = onnx.load("check.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %input[FLOAT, batch_sizex128]
) initializers (
  %enc_block.fc1.weight[FLOAT, 48x1024]
  %enc_block.fc1.bias[FLOAT, 48]
  %enc_block.fc2.weight[FLOAT, 1x48]
  %enc_block.fc2.bias[FLOAT, 1]
  %enc_block.conf_head.weight[FLOAT, 1x1024]
  %enc_block.conf_head.bias[FLOAT, 1]
  %dec_block.deconv_block1.deconv.weight[FLOAT, 512x256x3]
  %dec_block.deconv_block1.batchNorm.weight[FLOAT, 256]
  %dec_block.deconv_block1.batchNorm.bias[FLOAT, 256]
  %dec_block.deconv_block2.deconv.weight[FLOAT, 256x256x3]
  %dec_block.deconv_block3.deconv.weight[FLOAT, 256x256x3]
  %dec_block.deconv_block4.deconv.weight[FLOAT, 256x128x3]
  %dec_block.deconv_block4.batchNorm.weight[FLOAT, 128]
  %dec_block.deconv_block4.batchNorm.bias[FLOAT, 128]
  %dec_block.deconv_block5.deconv.weight[FLOAT, 128x64x3]
  %dec_block.deconv_block5.batchNorm.weight[FLOAT, 64]
  %dec_block.deconv_block5.batchNorm.bias[FLOAT, 64]
  %dec_block.deconv_block6.deconv.weight[FLOAT, 64x32x3]
  %dec_block.deconv_block6.ba

In [16]:
import onnxruntime as ort

ort_session = ort.InferenceSession("check.onnx", providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

outputs = ort_session.run(
    None,
    {"actual_input_1": np.random.randn(1, 128).astype(np.float32)},
)
print(outputs[0])

EP Error /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:114 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 710: device-side assert triggered ; GPU=0 ; hostname=pop-os ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=241 ; expr=cudaDeviceSynchronize(); 

 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecu

2023-07-31 21:09:51.187206836 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:614 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.


RuntimeError: /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:114 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 710: device-side assert triggered ; GPU=0 ; hostname=pop-os ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=241 ; expr=cudaDeviceSynchronize(); 



In [2]:
mod = Deconv_block()
ss = torch.jit.trace(mod, torch.rand(10, 64, 64))

TypeError: MaxUnpool1d.forward() missing 1 required positional argument: 'indices'

In [7]:
print(ss.code)


def forward(self,
    input_1D: Tensor) -> Tensor:
  relu = self.relu
  batchNorm = self.batchNorm
  deconv = self.deconv
  _0 = (batchNorm).forward((deconv).forward(input_1D, ), )
  return (relu).forward(_0, )



In [ ]:
print(torch.__version__)
torch.manual_seed(191009)

In [8]:
@torch.jit.script
def foo_sum(x,y):
    return x+ y

In [9]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
        

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return foo_sum(new_h, new_h)

In [10]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell2(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell2, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

In [ ]:
my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

In [11]:
my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print("tracd",traced_cell,"end")
traced_cell(x, h)

tracd MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
) end


tensor([[ 0.7430,  1.4186,  1.5269,  0.7439],
        [ 0.6736, -0.4245,  0.9019, -0.2510],
        [ 1.0948,  0.4671,  0.8218,  0.3636]], grad_fn=<AddBackward0>)

In [12]:
#print(traced_cell.graph)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tensor:
  linear = self.linear
  new_h = torch.tanh(torch.add((linear).forward(x, ), h))
  return __torch__.foo_sum(new_h, new_h, )



In [ ]:
my_cell = MyCell2(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

In [ ]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell2(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)
print(scripted_cell(x, h))#

In [ ]:

class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.script(MyCell2(scripted_gate))
        self.unpool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
    
    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.unpool(xs[i], h)
        return y, h

rnn_loop = torch.jit.script( MyRNNLoop())
print(rnn_loop.code)
print(rnn_loop.cell.code)

In [ ]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        temp=0
        for i in h:
            temp+=y
        return torch.relu(y)+temp

traced = torch.jit.script(WrapRNN())
print(traced.loop.cell.code)

In [2]:
class convBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batch = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.maxPool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1, return_indices=True)
    def forward(self, input_1D):
        input_1D = self.conv(input_1D)
        input_1D = self.relu(self.batch(input_1D))
        input_1D, indx_mat = self.maxPool(input_1D)

        return input_1D, indx_mat

In [3]:
class deconvBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.deconv = nn.ConvTranspose1d(64, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchNorm = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.unpool = nn.MaxUnpool1d(kernel_size=3, stride=1, padding=0)
    
    def forward(self, input_1D1, idmat):
        input_1D = self.unpool(input_1D1, idmat)
        input_1D = self.deconv(input_1D)
        # batch norm
        input_1D = self.batchNorm(input_1D)
        # relu 
        input_1D = self.relu(input_1D)
        return input_1D

In [4]:
class modelFull(nn.Module):
    def __init__(self):
        super().__init__()
        self.bll = convBlock()
        self.deconv = deconvBlock()
    def forward(self, x):
            xx, y = self.bll(x)
            xz = self.deconv(xx, y)
            return xz

In [14]:
torch.jit.trace(Spice_model2(), torch.rand(1,  128))

Spice_model2(
  original_name=Spice_model2
  (enc_block): Spice_Encoder(
    original_name=Spice_Encoder
    (conv_block1): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNorm1d(original_name=BatchNorm1d)
      (relu): ReLU(original_name=ReLU)
      (maxPool): MaxPool1d(original_name=MaxPool1d)
    )
    (conv_block2): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNorm1d(original_name=BatchNorm1d)
      (relu): ReLU(original_name=ReLU)
      (maxPool): MaxPool1d(original_name=MaxPool1d)
    )
    (conv_block3): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNorm1d(original_name=BatchNorm1d)
      (relu): ReLU(original_name=ReLU)
      (maxPool): MaxPool1d(original_name=MaxPool1d)
    )
    (conv_block4): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNo

In [6]:
s = modelFull()
summary(s, input_size=[2, 1, 128])

Layer (type:depth-idx)                   Output Shape              Param #
modelFull                                [2, 32, 66]               --
├─convBlock: 1-1                         [2, 64, 64]               --
│    └─Conv1d: 2-1                       [2, 64, 128]              192
│    └─BatchNorm1d: 2-2                  [2, 64, 128]              128
│    └─ReLU: 2-3                         [2, 64, 128]              --
│    └─MaxPool1d: 2-4                    [2, 64, 64]               --
├─deconvBlock: 1-2                       [2, 32, 66]               --
│    └─MaxUnpool1d: 2-5                  [2, 64, 66]               --
│    └─ConvTranspose1d: 2-6              [2, 32, 66]               6,144
│    └─BatchNorm1d: 2-7                  [2, 32, 66]               64
│    └─ReLU: 2-8                         [2, 32, 66]               --
Total params: 6,528
Trainable params: 6,528
Non-trainable params: 0
Total mult-adds (M): 0.86
Input size (MB): 0.00
Forward/backward pass size (MB): 0

In [ ]:
torch.jit.script(modelFull())

In [ ]:
spice = Spice_Decoder()

In [ ]:
torch.jit.script(spice)

In [ ]:
fs = 16000
med = MedleyDBLoader(fs )

In [ ]:
f1 = 1110.0644999999954 
f2 = -5.684341886080802e-14
dive = f1/f2
dive

In [ ]:
np.log2(dive)

In [ ]:
data_pd = pd.read_pickle('./CQT_data/MIR1k.pkl')

In [ ]:
nnp = np.load('./CQT_data/MedleyDB.npy')

In [ ]:
nnp = torch.toTe(nnp)

In [ ]:
data_pd.shape

In [ ]:
#data_pd = pd.DataFrame(data=nnp) 
train, val = train_test_split(data_pd, train_size=0.8, test_size=0.2, random_state=1)
print("train shape: ", train.shape)
train_batches = DataLoader(CQT_Dataset(data=train, mode='train'), batch_size=64, shuffle=True)
print("train_batch NUmber: ", len(train_batches))
diff, slice1, slice2, f0 = next(iter(train_batches))
print(f"diff batch shape: {diff.size()}")
print(f"slice1 batch shape: {slice1.size()}")
print(f"slice2 batch shape: {f0.size()}")

spice = Spice_model()


for b in train_batches:
    print("batch shape:", b[0].shape)
    pitch_diff, x_1, x_2, f0 = b
    x_1 = x_1.type(torch.FloatTensor)
    #print(x_1.shape, x_1.type())
    a, x, y = spice(x_1)
    print(a.size(), x.size(), y.size())

In [ ]:
xx=torch.arange(0, 25, 1)

In [ ]:
xx = xx.reshape((5,-1))

In [ ]:
xx.size()

In [ ]:
df = pd.read_pickle('./CQT_data/MIR1k.pkl')

In [ ]:
df.shape

In [ ]:
np.arange(0,25,1).reshape(5,5)[:,-1:]

In [ ]:
a = np.arange(0,15,1).reshape(5,3)
a = np.hstack((a, np.array([0.25,0.7,0,0.5,0]).reshape(-1,1)))
a

In [ ]:
df = pd.DataFrame(a)
df.shape

In [ ]:
# remove rows of cqt where label (last) column is zero
df.drop(df.loc[df.iloc[:, -1]==0].index, inplace=True)
df.shape

In [ ]:
np.min(df.iloc[:, -1])

In [3]:
torch.jit.script(Spice_model2())

RuntimeError: isIntList() INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/core/ivalue_inl.h":1938, please report a bug to PyTorch. Expected IntList but got Int

In [2]:
s = Spice_model2()
summary(s, input_size=[10, 128])

Layer (type:depth-idx)                   Output Shape              Param #
Spice_model2                             [10, 1]                   --
├─Spice_Encoder: 1-1                     [10, 1]                   --
│    └─Conv_block: 2-1                   [10, 64, 64]              --
│    │    └─Conv1d: 3-1                  [10, 64, 128]             192
│    │    └─BatchNorm1d: 3-2             [10, 64, 128]             128
│    │    └─ReLU: 3-3                    [10, 64, 128]             --
│    │    └─MaxPool1d: 3-4               [10, 64, 64]              --
│    └─Conv_block: 2-2                   [10, 128, 32]             --
│    │    └─Conv1d: 3-5                  [10, 128, 64]             24,576
│    │    └─BatchNorm1d: 3-6             [10, 128, 64]             256
│    │    └─ReLU: 3-7                    [10, 128, 64]             --
│    │    └─MaxPool1d: 3-8               [10, 128, 32]             --
│    └─Conv_block: 2-3                   [10, 256, 16]             --
│    │  

In [ ]:
for n, param in s.named_parameters():
    print(type(param), param.requires_grad)